In [1]:
import pandas as pd
import numpy as np
import time
import datetime
import os
import pickle
os.chdir('/home/valesco/Datasets/CFB/')
pd.set_option('display.max_columns', 50)
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set_context('poster')
sns.set_style('white')
sns.set_color_codes()
plot_kwds = {'alpha' : 0.5, 's' : 80, 'linewidths':0}
import hdbscan
from sklearn import decomposition
from bokeh.charts import Scatter
from bokeh.io import output_notebook, show

In [2]:
wr_df = pd.read_csv('wr_target_log_2012_2015.csv')
wr_df.head(15)

,gsis_id,play_id,player_id,team,player_id.1,last_name,first_name,team.1,position,birthdate,...,passing_att,passing_cmp,passing_cmp_air_yds,passing_incmp,passing_incmp_air_yds,receiving_rec,receiving_tar,receiving_tds,receiving_yac_yds,receiving_yds
0,2012090905,462,00-0026035,STL,00-0026035,Amendola,Danny,NE,WR,11/2/1985,...,0,0,0,0,0,1,1,0,3,15
1,2012090905,606,00-0026035,STL,00-0026035,Amendola,Danny,NE,WR,11/2/1985,...,0,0,0,0,0,1,1,0,5,17
2,2012090905,1031,00-0026035,STL,00-0026035,Amendola,Danny,NE,WR,11/2/1985,...,0,0,0,0,0,0,1,0,0,0
3,2012090905,2062,00-0026035,STL,00-0026035,Amendola,Danny,NE,WR,11/2/1985,...,0,0,0,0,0,1,1,0,3,9
4,2012090905,2121,00-0026035,STL,00-0026035,Amendola,Danny,NE,WR,11/2/1985,...,0,0,0,0,0,0,1,0,0,0
5,2012090905,2385,00-0026035,STL,00-0026035,Amendola,Danny,NE,WR,11/2/1985,...,0,0,0,0,0,0,1,0,0,0
6,2012090905,3285,00-0026035,STL,00-0026035,Amendola,Danny,NE,WR,11/2/1985,...,0,0,0,0,0,1,1,0,2,14
7,2012090905,3441,00-0026035,STL,00-0026035,Amendola,Danny,NE,WR,11/2/1985,...,0,0,0,0,0,0,1,0,0,0
8,2012090905,3776,00-0026035,STL,00-0026035,Amendola,Danny,NE,WR,11/2/1985,...,0,0,0,0,0,1,1,0,0,15
9,2012090907,560,00-0024324,PHI,00-0024324,Avant,Jason,KC,WR,4/20/1983,...,0,0,0,0,0,1,1,0,2,5


In [9]:
def convert_time(x):
    str_ls = x.split('/')
    new_ls = []
    for s in str_ls[:2]:
        if len(s) < 2:
            s = '0' + s
        new_ls.append(s)
        
    final_str = new_ls[0] + '/' + new_ls[1] + '/' + str_ls[2]    
       
    stamp = time.mktime(datetime.datetime.strptime(final_str, '%m/%d/%Y').timetuple())
    return stamp
    
wr_df['birthstamp'] = wr_df['birthdate'].apply(lambda x: convert_time(x))
wr_df.head()

,gsis_id,play_id,player_id,team,player_id.1,last_name,first_name,team.1,position,birthdate,...,passing_cmp,passing_cmp_air_yds,passing_incmp,passing_incmp_air_yds,receiving_rec,receiving_tar,receiving_tds,receiving_yac_yds,receiving_yds,birthstamp
0,2012090905,462,0026035,STL,00-0026035,Amendola,Danny,NE,WR,11/2/1985,...,0,0,0,0,1,1,0,3,15,499759200
1,2012090905,606,0026035,STL,00-0026035,Amendola,Danny,NE,WR,11/2/1985,...,0,0,0,0,1,1,0,5,17,499759200
2,2012090905,1031,0026035,STL,00-0026035,Amendola,Danny,NE,WR,11/2/1985,...,0,0,0,0,0,1,0,0,0,499759200
3,2012090905,2062,0026035,STL,00-0026035,Amendola,Danny,NE,WR,11/2/1985,...,0,0,0,0,1,1,0,3,9,499759200
4,2012090905,2121,0026035,STL,00-0026035,Amendola,Danny,NE,WR,11/2/1985,...,0,0,0,0,0,1,0,0,0,499759200


In [13]:
def time_line(df):
    week = float(df['week'])/18
    year = float(df['season_year'])
    return (year + week)

wr_df['timeline'] = wr_df.apply(time_line, axis = 1)
    

In [92]:
player_ls = np.unique(wr_df['player_id'].values)
time_ls = np.unique(wr_df['timeline'].values)

In [108]:
arr = np.empty((0, 304))

for p in player_ls:
    temp_df = wr_df.loc[wr_df['timeline'] < time_ls[3]]
    temp_df = temp_df.loc[temp_df['player_id'] == p]
    
    if len(temp_df) < 1:
        continue
    
    player_id = int(p)
    weight = int(temp_df.iloc[0]['weight'])
    height = int(temp_df.iloc[0]['height'])
    dob = float(temp_df.iloc[0]['birthstamp'])
    
    player_data = []
    player_data.extend([player_id, weight, height, dob])
    week1 = []
    week2 = []
    week3 = []
    for index, row in temp_df.iterrows():

        if row['timeline'] == time_ls[0]:
            w1_rec = int(row['receiving_rec'])
            w1_td = int(row['receiving_tds'])
            w1_yac = int(row['receiving_yac_yds'])
            w1_yds = int(row['receiving_yds'])
            
            week1.extend([w1_rec, w1_td, w1_yac, w1_yds])
            
        elif row['timeline'] == time_ls[1]:
            w2_rec = int(row['receiving_rec'])
            w2_td = int(row['receiving_tds'])
            w2_yac = int(row['receiving_yac_yds'])
            w2_yds = int(row['receiving_yds'])
            
            week2.extend([w2_rec, w2_td, w2_yac, w2_yds])
        elif row['timeline'] == time_ls[2]:
            w3_rec = int(row['receiving_rec'])
            w3_td = int(row['receiving_tds'])
            w3_yac = int(row['receiving_yac_yds'])
            w3_yds = int(row['receiving_yds'])
            
            week3.extend([w3_rec, w3_td, w3_yac, w3_yds])
        else:
            print("FAIL!!!!")
    
    week1.extend([0 for _ in range(100 - len(week1))])
    week2.extend([0 for _ in range(100 - len(week2))])        
    week3.extend([0 for _ in range(100 - len(week3))])  
    
    player_data.extend(week1)
    player_data.extend(week2)
    player_data.extend(week3)
    
    temp_arr = np.asarray(player_data)
    
    arr = np.append(arr, [temp_arr], axis = 0)
            

In [2]:

#f = open('wr_2012_array.pickle', 'wb')
#pickle.dump(arr, f)
#f.close()

arr = np.load('wr_2012_array.pickle')

In [129]:
clusterer = hdbscan.HDBSCAN(min_cluster_size = 5, gen_min_span_tree = True)
clusterer.fit(arr)

HDBSCAN(algorithm='best', allow_single_cluster=False, alpha=1.0,
    approx_min_span_tree=True, core_dist_n_jobs=4, gen_min_span_tree=True,
    leaf_size=40, memory=Memory(cachedir=None), metric='euclidean',
    min_cluster_size=5, min_samples=None, p=None)

In [161]:
labels = np.array(clusterer.labels_).reshape((74,1))

In [151]:
pca = decomposition.PCA(n_components = 2)
pca.fit(arr)
X = np.asarray(pca.transform(arr))

In [206]:
X_id = np.concatenate((arr[0:,0].reshape(74,1), X), axis = 1)
final = np.concatenate((labels, X_id), axis = 1)

In [180]:
results = pd.DataFrame( data = final[0:, 0:], 
                        index = final[0:, 1],
                        columns = ['label', 'player_num','X','y'])

In [232]:
results['player_id'] = results['player_num'].apply(lambda x: '00' + str(x))
results['player_id'] = results['player_id'].apply(lambda x: x.split('.')[0])

results.tail()

,index,label,player_num,X,y,player_id,full_name
69,29638,1,29638,9.454446e+07,585.860011,0029638,"Floyd,Michael"
70,29640,-1,29640,7.942086e+07,954.255997,0029640,"Quick,Brian"
71,29664,-1,29664,1.379137e+08,-438.869212,0029664,"Gordon,Josh"
72,29707,1,29707,9.825966e+07,564.876806,0029707,"Blackmon,Justin"
73,29708,1,29708,9.324846e+07,687.351523,0029708,"Wright,Kendall"


In [230]:
playas = np.unique(results['player_id'].values)

name_ls = []
for playa in playas:
    temp_df = wr_df.loc[wr_df['player_id'] == playa]
    name = temp_df['full_name'].iloc[0]
    name_ls.append(name)
    
results['full_name'] = pd.Series(name_ls, index = results.index)
results.reset_index(inplace = True)
    

In [277]:
output_notebook()
from bokeh.plotting import figure, ColumnDataSource
from bokeh.models import HoverTool

Loading BokehJS ...

In [264]:
def scatter_with_hover(df, x, y, fig=None, cols=None, name=None, marker = 'o',
                      fig_width = 600, fig_height = 600, color= 'label', **kwargs):
    

    source = ColumnDataSource(data = results)

    if fig is None:
        fig = figure(width=fig_width, height=fig_height, tools=['box_zoom', 'reset'])

    fig.scatter(df[x], df[y], source = source,  color = df[color], size = 10, **kwargs,
               fill_alpha = 1)

    hover = HoverTool(names = [name])

    if cols is None:
        hover.tooltips = [(c, '@' + c) for c in df.columns]
    else:
        hover.tooltips = [(c, '@' + c) for c in cols]

    hover.tooltips.append(('index', '$index'))
    fig.add_tools(hover)
    
    return fig

    #scat = Scatter(results, x = 'X', y = 'y', color = 'label')

    #show(scat)
    
show(scatter_with_hover(results, 'X', 'y',  color = 'label'))

In [278]:
scat = Scatter(results, x = 'X', y = 'y', color = 'label')


show(scat)

In [281]:
dist_mat = clusterer.probabilities_

In [283]:
clust1 = hdbscan.HDBSCAN(min_cluster_size = 5, gen_min_span_tree = True)
clusterer.fit(arr)

5

In [285]:
from sklearn.metrics.pairwise import pairwise_distances

In [286]:
dist_matrix = pairwise_distances(arr, n_jobs = 4)

In [288]:
clust1 = hdbscan.HDBSCAN(metric = 'precomputed')
clust1.fit(dist_matrix)
clust1.labels_

array([-1,  1, -1,  1, -1,  1,  1,  1,  1, -1,  1,  1,  1,  1,  1,  1, -1,
       -1,  1, -1, -1,  3,  3, -1,  3,  3,  3, -1,  3,  0,  0,  0,  0,  0,
        0,  3,  0,  3,  3,  3,  0,  0,  3,  3,  3,  0,  0,  0,  0,  0, -1,
        0,  0,  0, -1,  0,  0,  0,  0, -1,  0,  2, -1,  2,  0,  2,  2,  2,
        2,  2,  2, -1,  2,  2])

In [289]:
clust1.probabilities_

array([ 0.        ,  0.39105392,  0.        ,  0.39105392,  0.        ,
        0.43162451,  1.        ,  1.        ,  1.        ,  0.        ,
        0.5884197 ,  0.86359947,  0.44773397,  1.        ,  0.64844544,
        1.        ,  0.        ,  0.        ,  0.62409544,  0.        ,
        0.        ,  1.        ,  1.        ,  0.        ,  1.        ,
        0.83815946,  0.9263481 ,  0.        ,  0.91710071,  1.        ,
        1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
        0.91710071,  1.        ,  1.        ,  0.9263481 ,  0.96704806,
        1.        ,  1.        ,  1.        ,  0.76557972,  0.7791298 ,
        1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
        0.        ,  1.        ,  1.        ,  1.        ,  0.        ,
        1.        ,  1.        ,  1.        ,  1.        ,  0.        ,
        1.        ,  0.56962025,  0.        ,  1.        ,  1.        ,
        0.41666667,  1.        ,  1.        ,  1.        ,  1.  

In [291]:
dist_matrix.shape

(74, 74)